DATASET PREPARATION

In [ ]:
import json
import random

# File paths
data_file = 'dataset/processed_llama_output.jsonl'  # path to your JSONL file
train_file = 'dataset/train/train_data.jsonl'
val_file = 'dataset/val/val_data.jsonl'

# Step 1: Load the data from the JSONL file
with open(data_file, 'r') as f:
    data = [json.loads(line) for line in f]

# Step 2: Shuffle the data to ensure random distribution
random.shuffle(data)

# Step 3: Calculate the split sizes
train_size = int(len(data) * 0.8)
val_size = int(len(data) * 0.2)

# Step 4: Split the data into train, validation, and test sets
train_data = data[:train_size]
val_data = data[train_size:train_size + val_size]

# Step 5: Save the splits into separate JSONL files
def save_jsonl(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')

save_jsonl(train_data, train_file)
save_jsonl(val_data, val_file)

print(f"Data split into train, validation, and test sets.")
print(f"Train set: {train_size} records")
print(f"Validation set: {val_size} records")

TRAINING

In [ ]:
%%bash

# Set paths to the model, train, validation and test sets.
MODEL="./llama-3_1-8b-instruct-nemo_v1.0/llama3_1_8b_instruct.nemo"

TRAIN_DS="[dataset/train/train_data.jsonl]"
VALID_DS="[dataset/val/val_data.jsonl]"
TEST_DS="[dataset/test/test_data.jsonl]"
TEST_NAMES="[factcheck]"
SCHEME="lora"
TP_SIZE=1
PP_SIZE=1

rm -rf results
OUTPUT_DIR="./results/Meta-llama3.1-8B-Instruct-factgen"

torchrun --nproc_per_node=1 \
/opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
    exp_manager.exp_dir=${OUTPUT_DIR} \
    exp_manager.explicit_log_dir=${OUTPUT_DIR} \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    trainer.precision=bf16-mixed \
    trainer.val_check_interval=0.2 \
    trainer.max_steps=900 \
    model.megatron_amp_O2=True \
    ++model.mcore_gpt=True \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    model.micro_batch_size=1 \
    model.global_batch_size=32 \
    model.restore_from_path=${MODEL} \
    model.data.train_ds.file_names=${TRAIN_DS} \
    model.data.train_ds.concat_sampling_probabilities=[1.0] \
    model.data.validation_ds.file_names=${VALID_DS} \
    model.peft.peft_scheme=${SCHEME}

Pull your server to test deployment (I personally used NIM)

EXAMPLE OUTPUT:

In [ ]:
import requests
import json
url = 'your-host-url'
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# Example from the test set
prompt= """
        Task: Contrast the claim using the given text. 
        Claim: Mindfulness in education will universally improve academic scores. 
        [SEP] Text: Figure 1 summarizes the study design and the numbers of students assessed at each time point. Table 1 Demographics of intervention and control cohortFull size 
        tableConfirmation of reliability and validity of the questionnairesThe reliability and validity of the perceived stress (PSS-10) and and Awareness (MAAS) 
        Scales were confirmed by using all data sates that were returned to us (T1: n = 139, T3: n = 148). Internal consistencies were assessed by calculating 
        Cronbach\u2019s alpha for both scales (PSS and MAAS) and both time points. The resulting values ranged between 0. 851 and 0. 914 and thus proved reliable. 
        Validities were confirmed via confirmatory factor analyses and the results also showed a good fit (additional Table 1[see additional file 1]). Further descriptive statistics 
        for the questionnaires are provided in the additional Tables 2\u20135 [see additional file 1]. Mindfulness and perceived stress were inversely correlated. Our longitudinal analysis 
        of medical students over their first and third term revealed, that elevated mindfulness was paralleled by a reduction of perceived stress and vice versa. 
        Spearman rank correlation analyses performed for our study cohort revealed statistically significant correlations with coefficients of r = \u2212 0. 4594 at T1 
        (95% CI = \u2212 0. 5831 to \u2212 0. 3149, p < 0. 0001) and r = \u2212 0. 5300 at T3 (95% CI = \u2212 0. 6552 to \u2212 0. 3764, p < 0. 0001) (Fig. 2).
        Neither mindfulness nor perceived stress correlated with academic achievements in the first year at medical schoolWe here assessed whether mindfulness or 
        perceived stress were associated with exam results in first year medical students and therefore correlated the exam scores achieved at T1 with the scores obtained from the 
        simultaneously assessed stress and mindfulness scales, respectively. While there were no correlations between mindfulness and exam scores, there was a 
        negligible correlation between perceived stress and the results from the biology exam that yielded in a correlation coefficient of 0. 177 and a p-value of 0. 0456.
        The improvement of academic achievements following an intervention on MBSR was transientIn order to analyze whether our intervention teaching formal techniques of 
        MBSR impacted positively on the academic performance, we compared exam results between intervention and control cohorts. [SEP] JUSTIFY:
"""
data = {
    "model": "your fact-checking path",
    "prompt": prompt,
    "max_tokens": 500
}

response = requests.post(url, headers=headers, json=data)
response_data = response.json()

print(f"OUTPUT: {response_data['choices'][0]['text']}")

EXAMPLE OUTPUT: The text contrasts the claim by providing evidence that mindfulness in medical education did not correlate with academic achievements for first-year medical students. It states that while elevated mindfulness was associated with a reduction in perceived stress, it found no correlation with academic achievements, thereby challenging the idea that mindfulness leads to universally improved grades.